# Fitting cost-of-carry to market data

In this assignment, you will examine how well cost-of-carry model assumptions fit market data.

Use settlement prices on Thursday 2025-10-30 for all of the sections below, and use futures expirations going out two years.

1. How well does a single continuously compounded cost-of-carry rate fit gold futures? You may use the front month as a proxy for spot and check this question
only for the back months.

In [1]:
import datetime
from zoneinfo import ZoneInfo

import databento as db
import numpy as np
import plotly.express as px

from finm37000 import (
    get_all_legs_on,
    get_databento_api_key,
    temp_env,
)

with temp_env(DATABENTO_API_KEY=get_databento_api_key()):
    client = db.Historical()


In [2]:
date = datetime.date(2025, 10, 30)
gold_stats, gold_legs = get_all_legs_on(client, date, "GC.FUT")

In [3]:

seconds_per_year = 365.25 * 24 * 60 * 60
settlement_time = datetime.time(13, 30)
date_tz = datetime.datetime.combine(
    date, settlement_time, tzinfo=ZoneInfo("America/Chicago")
)
november_2027 = datetime.datetime(2027, 11, 1, tzinfo=ZoneInfo("America/Chicago"))
two_years = (
    gold_stats[gold_stats["expiration"] < november_2027]
    .xs(date)
    .sort_values("expiration")
)
front_settle = two_years["Settlement price"].iloc[0]
front_expiration = two_years["expiration"].iloc[0]
two_years["front_delta_to_expiration"] = two_years["expiration"] - front_expiration
two_years["front_to_expiration"] = (
    two_years["front_delta_to_expiration"].dt.total_seconds() / seconds_per_year
)
two_years["relative_to_front"] = two_years["Settlement price"] / front_settle
two_years["adjusted_cost_of_carry"] = (
    np.log(two_years["relative_to_front"]) / two_years["front_to_expiration"]
)
two_years

,Settlement price,Cleared volume,Open interest,expiration,front_delta_to_expiration,front_to_expiration,relative_to_front,adjusted_cost_of_carry
Symbol,,,,,,,,
GCX5,4001.3,NaN,NaN,2025-11-25 18:30:00+00:00,0 days 00:00:00,0.000000,1.000000,NaN
GCZ5,4015.9,NaN,NaN,2025-12-29 18:30:00+00:00,34 days 00:00:00,0.093087,1.003649,0.039127
GCF6,4033.0,NaN,NaN,2026-01-28 18:30:00+00:00,64 days 00:00:00,0.175222,1.007922,0.045035
GCG6,4049.3,NaN,NaN,2026-02-25 18:30:00+00:00,92 days 00:00:00,0.251882,1.011996,0.047342
GCH6,4063.7,NaN,NaN,2026-03-27 17:30:00+00:00,121 days 23:00:00,0.333904,1.015595,0.046344
GCJ6,4079.0,NaN,NaN,2026-04-28 17:30:00+00:00,153 days 23:00:00,0.421515,1.019419,0.045627
GCK6,4093.5,NaN,NaN,2026-05-27 17:30:00+00:00,182 days 23:00:00,0.500913,1.023043,0.045479
GCM6,4109.0,NaN,NaN,2026-06-26 17:30:00+00:00,212 days 23:00:00,0.583048,1.026916,0.045554
GCN6,4123.8,NaN,NaN,2026-07-29 17:30:00+00:00,245 days 23:00:00,0.673397,1.030615,0.044782


In [4]:
px.scatter(two_years, x="expiration", y="adjusted_cost_of_carry")

A constant storage rate does not seem like a good fit here. At best, there is a linear time trend from February onwards. The earlier expirations are
clearly outside this trend.

2. Unlike gold, the usefulness of crude should affect the spot-futures relationship through the convenience yield, so rather than looking directly
at the cost-of-carry, look at the cost-of-carry minus the convenience yield.
How well would a constant rate of this convenience-adjusted cost-of-carry fit the crude curve?
Note that crude spot prices are available from https://www.eia.gov/dnav/pet/hist/RWTCD.htm

In [5]:
crude_stats, crude_legs = get_all_legs_on(client, date, "CL.FUT")

In [6]:
crude_spot = 61.36  # from eia.gov
settlement_time = datetime.time(14, 30)
date_tz = datetime.datetime.combine(
    date, settlement_time, tzinfo=ZoneInfo("America/Chicago")
)
november_2027 = datetime.datetime(2027, 11, 1, tzinfo=ZoneInfo("America/Chicago"))
two_years = (
    crude_stats[crude_stats["expiration"] < november_2027]
    .xs(date)
    .sort_values("expiration")
)
front_settle = two_years["Settlement price"].iloc[0]
front_expiration = two_years["expiration"].iloc[0]
two_years["timedelta_to_expiration"] = two_years["expiration"] - date_tz
two_years["time_to_expiration"] = (
    two_years["timedelta_to_expiration"].dt.total_seconds() / seconds_per_year
)
two_years["front_delta_to_expiration"] = two_years["expiration"] - front_expiration
two_years["relative_to_spot"] = two_years["Settlement price"] / crude_spot
two_years["adjusted_cost_of_carry"] = (
    np.log(two_years["relative_to_spot"]) / two_years["time_to_expiration"]
)

two_years

,Settlement price,Cleared volume,Open interest,expiration,timedelta_to_expiration,time_to_expiration,front_delta_to_expiration,relative_to_spot,adjusted_cost_of_carry
Symbol,,,,,,,,,
CLZ5,60.57,NaN,NaN,2025-11-20 19:30:00+00:00,21 days 00:00:00,0.057495,0 days 00:00:00,0.987125,-0.225384
CLF6,60.20,NaN,NaN,2025-12-19 19:30:00+00:00,50 days 00:00:00,0.136893,29 days 00:00:00,0.981095,-0.139422
CLG6,59.91,NaN,NaN,2026-01-20 19:30:00+00:00,82 days 00:00:00,0.224504,61 days 00:00:00,0.976369,-0.106523
CLH6,59.79,NaN,NaN,2026-02-20 19:30:00+00:00,113 days 00:00:00,0.309377,92 days 00:00:00,0.974413,-0.083780
CLJ6,59.78,NaN,NaN,2026-03-20 18:30:00+00:00,140 days 23:00:00,0.385923,119 days 23:00:00,0.974250,-0.067596
CLK6,59.85,NaN,NaN,2026-04-21 18:30:00+00:00,172 days 23:00:00,0.473534,151 days 23:00:00,0.975391,-0.052619
CLM6,59.92,NaN,NaN,2026-05-19 18:30:00+00:00,200 days 23:00:00,0.550194,179 days 23:00:00,0.976532,-0.043163
CLN6,59.97,NaN,NaN,2026-06-22 18:30:00+00:00,234 days 23:00:00,0.643281,213 days 23:00:00,0.977347,-0.035620
CLQ6,59.98,NaN,NaN,2026-07-21 18:30:00+00:00,263 days 23:00:00,0.722679,242 days 23:00:00,0.977510,-0.031476


In [7]:
px.scatter(two_years, x="expiration", y="adjusted_cost_of_carry")

Like gold, this is not going to be well-modeled by a constant cost-of-carry minus convenience yield. However, the shape is more regular,
and there is a clearer shape that could be modeled. In practice, you could bring more data to bear such as a non-constant interest
rate model, data on storage costs, and supply-and-demand modeling to gauge the convenience yield.

3. How well does a single continuously compounded cost-of-carry minus convenience yield rate fit corn futures? You may use the front month as a proxy for spot and check this question
only for the back months.

In [8]:
corn_stats, corn_legs = get_all_legs_on(client, date, "ZC.FUT")

In [9]:
november_2027 = datetime.datetime(2027, 11, 1, tzinfo=ZoneInfo("America/Chicago"))
two_years = (
    corn_stats[corn_stats["expiration"] < november_2027]
    .xs(date)
    .sort_values("expiration")
)
front_settle = two_years["Settlement price"].iloc[0]
front_expiration = two_years["expiration"].iloc[0]
two_years["front_delta_to_expiration"] = two_years["expiration"] - front_expiration
two_years["front_to_expiration"] = (
    two_years["front_delta_to_expiration"].dt.total_seconds() / seconds_per_year
)
two_years["relative_to_front"] = two_years["Settlement price"] / front_settle
two_years["adjusted_cost_of_carry"] = (
    np.log(two_years["relative_to_front"]) / two_years["front_to_expiration"]
)
two_years

,Settlement price,Cleared volume,Open interest,expiration,front_delta_to_expiration,front_to_expiration,relative_to_front,adjusted_cost_of_carry
Symbol,,,,,,,,
ZCZ5,430.25,NaN,NaN,2025-12-12 18:01:00+00:00,0 days 00:00:00,0.000000,1.000000,NaN
ZCH6,443.75,NaN,NaN,2026-03-13 17:01:00+00:00,90 days 23:00:00,0.249030,1.031377,0.124061
ZCK6,452.25,NaN,NaN,2026-05-14 17:01:00+00:00,152 days 23:00:00,0.418777,1.051133,0.119082
ZCN6,458.75,NaN,NaN,2026-07-14 17:01:00+00:00,213 days 23:00:00,0.585786,1.066241,0.109492
ZCU6,453.50,NaN,NaN,2026-09-14 17:01:00+00:00,275 days 23:00:00,0.755533,1.054038,0.069658
ZCZ6,464.50,NaN,NaN,2026-12-14 18:01:00+00:00,367 days 00:00:00,1.004791,1.079605,0.076230
ZCH7,477.00,NaN,NaN,2027-03-12 18:01:00+00:00,455 days 00:00:00,1.245722,1.108658,0.082803
ZCK7,483.25,NaN,NaN,2027-05-14 17:01:00+00:00,517 days 23:00:00,1.418093,1.123184,0.081918
ZCN7,485.75,NaN,NaN,2027-07-14 17:01:00+00:00,578 days 23:00:00,1.585102,1.128995,0.076543


In [10]:
px.scatter(two_years, x="expiration", y="adjusted_cost_of_carry")

Convenience-adjusted cost-of-carry is clearly not constant across expirations. Moreover, there is a seasonality to the pattern, which
any model would need to account for.